In [1]:
from sqlalchemy.engine import Engine, Connection
from sqlalchemy import create_engine
from os import getenv
from typing import List, Dict, Optional
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
import numpy as np
from numpy import ndarray
import pytz

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
try:
    from dotenv import load_dotenv

    load_dotenv()
except:
    print('No ".env" file or python-dotenv not installed... Using default env variables...')

In [4]:
def db_connection() -> Engine:
    dbname: Optional[str] = getenv('POSTGRES_DB_NAME')
    host: Optional[str] = getenv('POSTGRES_HOST')
    user: Optional[str] = getenv('POSTGRES_USERNAME')
    password: Optional[str] = getenv('POSTGRES_PASSWORD')
    port: Optional[str] = getenv('POSTGRES_PORT')
        
    postgres_str: str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    
    engine: Engine = create_engine(postgres_str)
    
    return engine

# WHAT WE HAVE TO QUERY

let's start by define date intervals

In [5]:
startDate: str = (datetime.now(pytz.timezone('UTC')) - dt.timedelta(hours=168))
endDate: str = datetime.now(pytz.timezone('UTC'))
startDate, endDate 

(datetime.datetime(2021, 5, 29, 16, 35, 28, 917975, tzinfo=<UTC>),
 datetime.datetime(2021, 6, 5, 16, 35, 28, 918979, tzinfo=<UTC>))

In [6]:
targetDate: datetime = endDate + dt.timedelta(hours=12)
targetDates: List[datetime] = [endDate + dt.timedelta(hours=i) for i in range(1,13)]

CONVERT INTO STRINGS AND PUSHED TO THE START OF THE HOUR (xx:00:00)

In [7]:
start: str = startDate.strftime("%Y-%m-%d %H:00:00")
end: str = endDate.strftime("%Y-%m-%d %H:00:00")
targets: List[str] = [date.strftime("%Y-%m-%d %H:00:00") for date in targetDates]

In [10]:
start, end

('2021-05-29 16:00:00', '2021-06-05 16:00:00')

In [8]:
targets

['2021-06-05 17:00:00',
 '2021-06-05 18:00:00',
 '2021-06-05 19:00:00',
 '2021-06-05 20:00:00',
 '2021-06-05 21:00:00',
 '2021-06-05 22:00:00',
 '2021-06-05 23:00:00',
 '2021-06-06 00:00:00',
 '2021-06-06 01:00:00',
 '2021-06-06 02:00:00',
 '2021-06-06 03:00:00',
 '2021-06-06 04:00:00']

# QUERY OBSERVED INPUTS

where are they?

table: "meteomatics_weather" (for start to end - 1)

table: "meteomatics_forecast_weather" (for end)

In [38]:
query1: str = "SELECT * FROM meteomatics_weather WHERE timestamp_utc between '{}' and '{}'"
observed_df1: DataFrame = pd.read_sql_query(query1.format(start, end), con=db_connection())
observed_df1.drop(['id'], axis=1, inplace=True)

In [39]:
observed_df1

,plant_code,timestamp_utc,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,diffuse_rad_w,direct_rad_w,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,wind_dir_mean_100m_1h_d,wind_dir_mean_10m_1h_d,wind_gusts_10m_1h_ms,wind_gusts_100m_1h_ms,wind_gusts_100m_ms,wind_gusts_10m_ms,low_cloud_cover_p,medium_cloud_cover_p,high_cloud_cover_p
0,UP_PRCLCDPLRM_1,2021-05-31 12:00:00,12.8,24.5,1016.4,956.8,0.0,70.8,0.4,3.0,3.6,348.9,354.9,4.3,5.0,5.0,4.3,13.3,12.1,41.3
1,UP_PRCLCDMZRD_1,2021-05-31 12:00:00,14.9,27.1,1015.7,997.4,0.0,162.3,821.5,3.1,4.1,271.9,268.4,4.4,5.6,5.6,4.4,5.5,33.0,82.0
2,UP_PRCLCDPRZZ_1,2021-05-31 12:00:00,11.4,24.6,1016.1,948.7,0.0,267.6,8.0,3.2,4.0,315.4,318.9,4.4,5.3,5.2,4.2,18.5,17.5,35.2
3,UP_PRCLCMINEO_1,2021-05-31 12:00:00,12.6,24.5,1015.2,971.4,0.0,278.8,364.3,1.3,1.7,279.3,282.4,3.4,3.8,1.0,0.8,46.1,75.7,13.4
4,UP_PEPIZZA_1,2021-05-31 12:00:00,11.1,23.8,1016.1,991.7,0.0,326.6,237.4,2.3,3.6,280.4,282.0,5.1,5.8,4.5,3.9,9.3,32.5,52.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,UP_PEPIZZA_1,2021-06-05 16:00:00,14.4,22.6,1021.5,997.2,0.0,117.3,155.9,4.9,7.3,263.3,262.1,8.8,9.8,9.8,8.8,0.0,15.2,5.5
1121,UP_MPNTLCSMBC_1,2021-06-05 16:00:00,5.1,19.3,1022.0,921.3,0.0,123.6,112.0,8.2,8.4,287.3,286.2,11.0,11.3,11.3,11.0,0.0,47.9,15.8
1122,UP_MPNTLCDMRN_1,2021-06-05 16:00:00,12.8,24.1,1022.0,959.9,0.0,66.0,239.8,1.5,1.7,20.9,25.2,2.4,2.6,2.3,2.1,0.0,1.3,0.3
1123,ESAPRO_P263,2021-06-05 16:00:00,9.6,30.2,1018.3,1001.3,0.0,98.1,118.7,3.3,4.5,286.9,289.3,5.2,6.1,6.1,5.2,0.0,49.1,0.4


In [40]:
query2: str = "SELECT * FROM meteomatics_forecast_weather WHERE forecast_timestamp_utc = '{}' and timestamp_query_utc = '{}'"
observed_df2: DataFrame = pd.read_sql_query(query2.format(end, end), con=db_connection())
observed_df2.drop(['id','timestamp_query_utc'], axis=1, inplace=True)
observed_df2.rename(columns={'forecast_timestamp_utc': 'timestamp_utc'}, inplace=True)

In [41]:
observed_df2

,plant_code,timestamp_utc,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,diffuse_rad_w,direct_rad_w,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,wind_dir_mean_100m_1h_d,wind_dir_mean_10m_1h_d,wind_gusts_10m_1h_ms,wind_gusts_100m_1h_ms,wind_gusts_100m_ms,wind_gusts_10m_ms,low_cloud_cover_p,medium_cloud_cover_p,high_cloud_cover_p
0,UP_PRCLCDPLRM_1,2021-06-05 16:00:00,13.1,24.7,1022.0,962.5,0.00,88.2,1.7,1.7,1.9,20.0,23.7,2.6,2.9,2.5,2.2,0.0,1.0,0.0
1,UP_PRCLCDMZRD_1,2021-06-05 16:00:00,7.8,25.2,1021.6,1003.3,0.18,58.8,297.8,2.8,3.7,169.5,171.1,4.8,5.2,5.2,4.8,0.0,1.3,4.9
2,UP_PRCLCDPRZZ_1,2021-06-05 16:00:00,11.4,24.6,1022.1,954.7,0.00,94.6,2.0,1.0,1.3,249.2,253.6,2.2,2.8,1.2,1.0,0.4,0.8,0.0
3,UP_PRCLCMINEO_1,2021-06-05 16:00:00,8.7,25.6,1021.1,977.2,0.00,53.7,271.6,5.2,6.6,235.3,233.3,7.1,8.5,8.1,6.9,0.0,20.4,85.6
4,UP_PEPIZZA_1,2021-06-05 16:00:00,14.4,22.6,1021.5,997.2,0.00,117.3,155.9,4.9,7.3,263.3,262.1,8.8,9.8,9.8,8.8,0.0,15.2,5.5
5,UP_MPNTLCSMBC_1,2021-06-05 16:00:00,5.1,19.3,1022.0,921.3,0.00,123.6,112.0,8.2,8.4,287.3,286.2,11.0,11.3,11.3,11.0,0.0,47.9,15.8
6,UP_MPNTLCDMRN_1,2021-06-05 16:00:00,12.8,24.1,1022.0,959.9,0.00,66.0,239.8,1.5,1.7,20.9,25.2,2.4,2.6,2.3,2.1,0.0,1.3,0.3
7,ESAPRO_P263,2021-06-05 16:00:00,9.6,30.2,1018.3,1001.3,0.00,98.1,118.7,3.3,4.5,286.9,289.3,5.2,6.1,6.1,5.2,0.0,49.1,0.4
8,ESAPRO_P259,2021-06-05 16:00:00,16.5,26.3,1017.5,1017.3,0.00,86.5,230.2,0.9,1.1,64.8,69.1,2.3,2.5,0.8,0.8,3.7,41.2,34.3


In [42]:
observed_df2.dtypes

plant_code                            object
timestamp_utc                 datetime64[ns]
dew_point_2m_C                       float64
temperature_2m_C                     float64
msl_pressure_hPa                     float64
sfc_pressure_hPa                     float64
precipitation_1h_mm                  float64
diffuse_rad_w                        float64
direct_rad_w                         float64
wind_speed_mean_10m_1h_ms            float64
wind_speed_mean_100m_1h_ms           float64
wind_dir_mean_100m_1h_d              float64
wind_dir_mean_10m_1h_d               float64
wind_gusts_10m_1h_ms                 float64
wind_gusts_100m_1h_ms                float64
wind_gusts_100m_ms                   float64
wind_gusts_10m_ms                    float64
low_cloud_cover_p                    float64
medium_cloud_cover_p                 float64
high_cloud_cover_p                   float64
dtype: object

CONCAT THE TWO OBSERVED DF

In [43]:
farm_list: List = ['UP_PRCLCDPLRM_1',
'UP_PRCLCDMZRD_1',
'UP_PRCLCDPRZZ_1',
'UP_PRCLCMINEO_1',
'UP_PEPIZZA_1',
'UP_MPNTLCSMBC_1',
'UP_MPNTLCDMRN_1']

In [44]:
observed_df: DataFrame = pd.concat([observed_df1, observed_df2], axis=0, ignore_index=True)
observed_df: DataFrame = observed_df.sort_values(by=['timestamp_utc','plant_code'], ascending=True, ignore_index=True)
observed_df: DataFrame = observed_df[observed_df['plant_code'].isin(farm_list)]

In [45]:
observed_df

,plant_code,timestamp_utc,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,diffuse_rad_w,direct_rad_w,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,wind_dir_mean_100m_1h_d,wind_dir_mean_10m_1h_d,wind_gusts_10m_1h_ms,wind_gusts_100m_1h_ms,wind_gusts_100m_ms,wind_gusts_10m_ms,low_cloud_cover_p,medium_cloud_cover_p,high_cloud_cover_p
2,UP_MPNTLCDMRN_1,2021-05-31 12:00:00,13.1,24.0,1016.7,954.3,0.0,330.5,246.5,2.7,3.2,359.3,4.4,3.9,4.6,4.6,3.9,11.7,8.6,43.1
3,UP_MPNTLCSMBC_1,2021-05-31 12:00:00,8.5,18.4,1016.3,915.6,0.0,147.5,746.5,5.8,6.0,292.1,291.8,8.0,8.3,7.4,7.0,7.5,13.7,44.6
4,UP_PEPIZZA_1,2021-05-31 12:00:00,11.1,23.8,1016.1,991.7,0.0,326.6,237.4,2.3,3.6,280.4,282.0,5.1,5.8,4.5,3.9,9.3,32.5,52.1
5,UP_PRCLCDMZRD_1,2021-05-31 12:00:00,14.9,27.1,1015.7,997.4,0.0,162.3,821.5,3.1,4.1,271.9,268.4,4.4,5.6,5.6,4.4,5.5,33.0,82.0
6,UP_PRCLCDPLRM_1,2021-05-31 12:00:00,12.8,24.5,1016.4,956.8,0.0,70.8,0.4,3.0,3.6,348.9,354.9,4.3,5.0,5.0,4.3,13.3,12.1,41.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,UP_PRCLCDPLRM_1,2021-06-05 16:00:00,13.1,24.7,1022.0,962.5,0.0,88.2,1.7,1.7,1.9,20.0,23.7,2.6,2.9,2.5,2.2,0.0,1.0,0.0
1130,UP_PRCLCDPRZZ_1,2021-06-05 16:00:00,11.4,24.6,1022.1,954.7,0.0,94.6,2.0,1.0,1.3,249.2,253.6,2.2,2.8,1.2,1.0,0.4,0.8,0.0
1131,UP_PRCLCDPRZZ_1,2021-06-05 16:00:00,11.4,24.6,1022.1,954.7,0.0,94.6,2.0,1.0,1.3,249.2,253.6,2.2,2.8,1.2,1.0,0.4,0.8,0.0
1132,UP_PRCLCMINEO_1,2021-06-05 16:00:00,8.7,25.6,1021.1,977.2,0.0,53.7,271.6,5.2,6.6,235.3,233.3,7.1,8.5,8.1,6.9,0.0,20.4,85.6


# QUERY KNOWN INPUTS (FORECASTS)

where are they?

table: "meteomatics_forecast_weather" 

In [56]:
query_fore: str = "SELECT * FROM meteomatics_forecast_weather WHERE forecast_timestamp_utc between '{}' and '{}'"
known_df: DataFrame = pd.read_sql_query(query_fore.format(targets[0], targets[-1]), con=db_connection())
known_df.drop(['id', 'timestamp_query_utc'], axis=1, inplace=True)
known_df: DataFrame = known_df.sort_values(by=['forecast_timestamp_utc','plant_code'], ascending=True, ignore_index=True)
known_df: DataFrame = known_df[known_df['plant_code'].isin(farm_list)]

In [57]:
known_df

,plant_code,forecast_timestamp_utc,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,diffuse_rad_w,direct_rad_w,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,wind_dir_mean_100m_1h_d,wind_dir_mean_10m_1h_d,wind_gusts_10m_1h_ms,wind_gusts_100m_1h_ms,wind_gusts_100m_ms,wind_gusts_10m_ms,low_cloud_cover_p,medium_cloud_cover_p,high_cloud_cover_p
48,UP_MPNTLCDMRN_1,2021-06-05 17:00:00,12.0,23.5,1021.0,959.1,0.0,82.9,81.7,1.1,1.5,289.1,304.9,2.1,2.9,2.9,2.1,0.0,0.4,71.8
49,UP_MPNTLCDMRN_1,2021-06-05 17:00:00,12.0,23.5,1021.0,959.1,0.0,82.9,81.7,1.1,1.5,289.1,304.9,2.1,2.9,2.9,2.1,0.0,0.4,71.8
50,UP_MPNTLCDMRN_1,2021-06-05 17:00:00,12.0,23.5,1021.0,959.1,0.0,82.9,81.7,1.1,1.5,289.1,304.9,2.1,2.9,2.9,2.1,0.0,0.4,71.8
51,UP_MPNTLCDMRN_1,2021-06-05 17:00:00,12.0,23.5,1021.0,959.1,0.0,82.9,81.7,1.1,1.5,289.1,304.9,2.1,2.9,2.9,2.1,0.0,0.4,71.8
52,UP_MPNTLCDMRN_1,2021-06-05 17:00:00,12.0,23.5,1021.0,959.1,0.0,82.9,81.7,1.1,1.5,289.1,304.9,2.1,2.9,2.9,2.1,0.0,0.4,71.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,UP_PRCLCMINEO_1,2021-06-06 04:00:00,2.6,17.9,1020.2,976.1,0.0,12.9,2.9,2.3,4.2,268.7,247.5,5.6,6.1,6.1,5.6,0.0,29.4,6.2
2093,UP_PRCLCMINEO_1,2021-06-06 04:00:00,2.6,17.9,1020.2,976.1,0.0,12.9,2.9,2.3,4.2,268.7,247.5,5.6,6.1,6.1,5.6,0.0,29.4,6.2
2094,UP_PRCLCMINEO_1,2021-06-06 04:00:00,2.6,17.9,1020.2,976.1,0.0,12.9,2.9,2.3,4.2,268.7,247.5,5.6,6.1,6.1,5.6,0.0,29.4,6.2
2095,UP_PRCLCMINEO_1,2021-06-06 04:00:00,2.6,17.9,1020.2,976.1,0.0,12.9,2.9,2.3,4.2,268.7,247.5,5.6,6.1,6.1,5.6,0.0,29.4,6.2


# WE ONLY HAVE DATA STARTING FROM THE 31/05...GOTTA WAIT FEW DAYS